Two ideas to work on
1) Train sentence transformers
2) Find a legal courpus for training for embedding

In [12]:
import pandas as pd
from pandas_profiling import ProfileReport
import os

In [122]:
pip install rake-nltk

  Using cached rake_nltk-1.0.4-py2.py3-none-any.whl



In [135]:
pip install gensim

In [13]:
# df = pd.read_csv(r'semantic-segmentation-master\semantic-segmentation-master\data\text\2007_C_121.txt',sep='\t',header=None)
# df.columns = ['sentence','label']

In [18]:
folder = r'dataset\Task1-20210907T155244Z-001\Task1'
df = None
for f in os.listdir(folder):
    filepath=os.path.join(folder,f)
    print(filepath)
    if df is None:
        df = pd.read_csv(filepath,sep='\t',header=None)
    else:
        new_df = pd.read_csv(filepath,sep='\t',header=None)
        df = pd.concat([df,new_df])

df.columns = ['sentence','label']

dataset\Task1-20210907T155244Z-001\Task1\d_1.txt
dataset\Task1-20210907T155244Z-001\Task1\d_10.txt
dataset\Task1-20210907T155244Z-001\Task1\d_11.txt
dataset\Task1-20210907T155244Z-001\Task1\d_12.txt
dataset\Task1-20210907T155244Z-001\Task1\d_13.txt
dataset\Task1-20210907T155244Z-001\Task1\d_14.txt
dataset\Task1-20210907T155244Z-001\Task1\d_15.txt
dataset\Task1-20210907T155244Z-001\Task1\d_16.txt
dataset\Task1-20210907T155244Z-001\Task1\d_17.txt
dataset\Task1-20210907T155244Z-001\Task1\d_18.txt
dataset\Task1-20210907T155244Z-001\Task1\d_19.txt
dataset\Task1-20210907T155244Z-001\Task1\d_2.txt
dataset\Task1-20210907T155244Z-001\Task1\d_20.txt
dataset\Task1-20210907T155244Z-001\Task1\d_21.txt
dataset\Task1-20210907T155244Z-001\Task1\d_22.txt
dataset\Task1-20210907T155244Z-001\Task1\d_23.txt
dataset\Task1-20210907T155244Z-001\Task1\d_24.txt
dataset\Task1-20210907T155244Z-001\Task1\d_25.txt
dataset\Task1-20210907T155244Z-001\Task1\d_26.txt
dataset\Task1-20210907T155244Z-001\Task1\d_27.txt
da

In [62]:
df

,sentence,label
0,"One Lakshminarayana Iyer, a Hindu Brahmin, who...",Facts
1,Ramalakshmi had married the plaintiff and had ...,Facts
2,"They were all alive in December, 1924, when La...",Facts
3,Before his death he executed a will on 16th No...,Ratio of the decision
4,By this will he gave the following directions ...,Facts
...,...,...
74,In these circumstances it would be fair and pr...,Ratio of the decision
75,If and when he prefers a claim to this particu...,Ratio of the decision
76,"In this view of the matter,we feel that the Hi...",Ratio of the decision
77,"The appeal is,therefore,dismissed with costs",Ruling by Present Court


In [123]:
from rake_nltk import Rake
from nltk.corpus import stopwords 
r = Rake()


In [124]:
sen_list = df['sentence'].tolist()
text = ".".join(map(str,sen_list))
a=r.extract_keywords_from_text(text)
b=r.get_ranked_phrases()
c=r.get_ranked_phrases_with_scores()

In [139]:
import re
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

In [140]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [141]:
# Convert to list
data = df.sentence.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', str(sent)) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", str(sent)) for sent in data]

pprint(data[:1])

['One Lakshminarayana Iyer, a Hindu Brahmin, who owned considerable properties '
 'in the Tirunelveli district, died on 13th December, 1924, leaving him '
 'surviving a widow Ranganayaki, and a married daughter Ramalakshmi']


In [142]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['one', 'lakshminarayana', 'iyer', 'hindu', 'brahmin', 'who', 'owned', 'considerable', 'properties', 'in', 'the', 'tirunelveli', 'district', 'died', 'on', 'th', 'december', 'leaving', 'him', 'surviving', 'widow', 'ranganayaki', 'and', 'married', 'daughter', 'ramalakshmi']]


In [143]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['one', 'lakshminarayana', 'iyer', 'hindu', 'brahmin', 'who', 'owned', 'considerable', 'properties', 'in', 'the', 'tirunelveli', 'district', 'died', 'on', 'th_december', 'leaving', 'him', 'surviving', 'widow', 'ranganayaki', 'and', 'married', 'daughter', 'ramalakshmi']


In [144]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [145]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['own', 'considerable', 'property', 'district', 'die', 'leave', 'survive', 'widow', 'married', 'daughter', 'ramalakshmi']]


In [146]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]]


In [147]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('considerable', 1),
  ('daughter', 1),
  ('die', 1),
  ('district', 1),
  ('leave', 1),
  ('married', 1),
  ('own', 1),
  ('property', 1),
  ('ramalakshmi', 1),
  ('survive', 1),
  ('widow', 1)]]

In [152]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [153]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.067*"right" + 0.040*"may" + 0.037*"also" + 0.033*"give" + 0.029*"person" '
  '+ 0.028*"provide" + 0.028*"officer" + 0.028*"therefore" + 0.027*"take" + '
  '0.026*"law"'),
 (1,
  '0.055*"article" + 0.029*"basis" + 0.027*"proprietary" + 0.026*"conclusion" '
  '+ 0.024*"liable" + 0.022*"available" + 0.021*"note" + 0.017*"process" + '
  '0.017*"company" + 0.017*"previous"'),
 (2,
  '0.076*"time" + 0.067*"include" + 0.046*"low" + 0.035*"even" + '
  '0.029*"product" + 0.029*"appoint" + 0.028*"allow" + 0.027*"first" + '
  '0.026*"point" + 0.022*"examination"'),
 (3,
  '0.091*"issue" + 0.090*"suit" + 0.060*"benefit" + 0.042*"thus" + '
  '0.037*"concern" + 0.033*"declare" + 0.033*"vest" + 0.031*"nature" + '
  '0.028*"stand" + 0.026*"submission"'),
 (4,
  '0.093*"section" + 0.068*"case" + 0.051*"period" + 0.036*"language" + '
  '0.029*"property" + 0.028*"accuse" + 0.025*"possession" + 0.024*"woman" + '
  '0.022*"income" + 0.022*"sub"'),
 (5,
  '0.135*"state" + 0.088*"respondent" + 0.05

In [63]:
test_folder = r'test\test-aila-task1-2021-20210907T070626Z-001\test-aila-task1-2021'
test_df = None
for f in os.listdir(test_folder):
    filepath=os.path.join(test_folder,f)
    print(filepath)
    if test_df is None:
        test_df = pd.read_csv(filepath,sep='\t',header=None)
    else:
        new_df = pd.read_csv(filepath,sep='\t',header=None)
        test_df = pd.concat([test_df,new_df])

# print(test_df)
test_df.columns = ['sentence','label']

test\test-aila-task1-2021-20210907T070626Z-001\test-aila-task1-2021\d1.txt
test\test-aila-task1-2021-20210907T070626Z-001\test-aila-task1-2021\d10.txt
test\test-aila-task1-2021-20210907T070626Z-001\test-aila-task1-2021\d2.txt
test\test-aila-task1-2021-20210907T070626Z-001\test-aila-task1-2021\d3.txt
test\test-aila-task1-2021-20210907T070626Z-001\test-aila-task1-2021\d4.txt
test\test-aila-task1-2021-20210907T070626Z-001\test-aila-task1-2021\d5.txt
test\test-aila-task1-2021-20210907T070626Z-001\test-aila-task1-2021\d6.txt
test\test-aila-task1-2021-20210907T070626Z-001\test-aila-task1-2021\d7.txt
test\test-aila-task1-2021-20210907T070626Z-001\test-aila-task1-2021\d8.txt
test\test-aila-task1-2021-20210907T070626Z-001\test-aila-task1-2021\d9.txt


In [64]:
test_df

,sentence,label
0,"This appeal, by special leave, by the appellan...",Ratio of the decision
1,The respondent was appointed by an order of th...,Facts
2,The category of the post to which she was appo...,Facts
3,"By an order dated February 16, 1967 of the Dep...",Facts
4,The respondent filed Civil Rule no. 206 of 196...,Facts
...,...,...
93,"Further, in various pleadings in the Title Sui...",Ratio of the decision
94,"From these facts, it is clear that the respond...",Ratio of the decision
95,"In view of the foregoing discussion, we do not...",Ruling by Present Court
96,"However, the parties are left to bear their ow...",Ruling by Present Court


In [20]:
prof = ProfileReport(df)
prof.to_file(output_file='output.html')

Summarize dataset:   0%|          | 0/16 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=747ddf5c935b585cf5f96e7550de1a00363403e1c04d94572d98f0ee96127d3f
  Stored in directory: c:\users\i515580\appdata\local\pip\cache\wheels\d1\c1\0f\faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers


In [22]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
sentences = df['sentence'].tolist()
sentence_embeddings = model.encode(sentences)

In [25]:
sentence_embeddings.shape

(11053, 384)

In [65]:
test_embeddings = model.encode(test_df['sentence'].tolist())
test_embeddings.shape

(644, 384)

In [31]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [32]:
label_encoder = LabelEncoder()  
integer_encoded = label_encoder.fit_transform(df['label'].tolist()) 

In [85]:
list(label_encoder.classes_)

['Argument',
 'Facts',
 'Precedent',
 'Ratio of the decision',
 'Ruling by Lower Court',
 'Ruling by Present Court',
 'Statute']

In [66]:
test_integer_encoded = label_encoder.transform(test_df['label'].tolist())

In [68]:
enc = OneHotEncoder(handle_unknown='ignore')
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)  
oh_labels = enc.fit_transform(integer_encoded)  
test_integer_encoded = test_integer_encoded.reshape(len(test_integer_encoded),1)
test_oh_labels=enc.transform(test_integer_encoded)

In [69]:
oh_labels

<11053x7 sparse matrix of type '<class 'numpy.float64'>'
	with 11053 stored elements in Compressed Sparse Row format>

In [36]:
from sklearn import svm
lin_clf = svm.LinearSVC()

In [38]:
lin_clf.fit(sentence_embeddings, integer_encoded)

C:\Users\I515580\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LinearSVC()

In [40]:
# dec = lin_clf.decision_function([[1]])
# dec.shape[1]

In [73]:
svm_predict=lin_clf.predict(test_embeddings)

In [82]:
from sklearn.metrics import precision_recall_fscore_support

In [75]:
precision_recall_fscore_support(test_integer_encoded, svm_predict, average='macro')

(0.6190681726396011, 0.4245606368594977, 0.4416240815874178, None)

In [76]:
precision_recall_fscore_support(test_integer_encoded, svm_predict, average='micro')

(0.5543478260869565, 0.5543478260869565, 0.5543478260869565, None)

In [78]:
precision_recall_fscore_support(test_integer_encoded, svm_predict, average='weighted')

(0.5860481352523278, 0.5543478260869565, 0.538704870480898, None)

In [94]:
precision_recall_fscore_support(test_integer_encoded, svm_predict, average=None)

(array([0.5625    , 0.52403846, 0.32432432, 0.60942761, 1.        ,
        0.88461538, 0.42857143]),
 array([0.16071429, 0.73154362, 0.43636364, 0.61355932, 0.08      ,
        0.58974359, 0.36      ]),
 array([0.25      , 0.61064426, 0.37209302, 0.61148649, 0.14814815,
        0.70769231, 0.39130435]),
 array([ 56, 149,  55, 295,  25,  39,  25], dtype=int64))

In [99]:
from sklearn.neural_network import MLPClassifier
simpleNN = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100,50, 7), random_state=1)

In [100]:
simpleNN.fit(sentence_embeddings, integer_encoded)

C:\Users\I515580\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\I515580\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(100, 50, 7), random_state=1,
              solver='lbfgs')

In [101]:
simpleNN_predict=simpleNN.predict(test_embeddings)

In [102]:
precision_recall_fscore_support(test_integer_encoded, simpleNN_predict, average='macro')

(0.4372340451632564, 0.38563614343632874, 0.39308916785033937, None)

In [104]:
pip install tensorflow_hub

Note: you may need to restart the kernel to use updated packages.


In [105]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys

In [117]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
module_path=r'C:\Users\I515580\Documents\universal-sentence-encoder_4'
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.load(module_path)
# with tf.compat.v1.Session() as session:

#   session.run([tf.compat.v1.global_variables_initializer(), tf.compat.v1.tables_initializer()])


In [119]:
sentences_embeddings_usc = embed(sentences)
test_embeddings_usc = embed(test_df['sentence'].tolist())

In [120]:
lin_clf_usc = svm.LinearSVC()
lin_clf_usc.fit(sentences_embeddings_usc, integer_encoded)
svm_predict_usc=lin_clf_usc.predict(test_embeddings_usc)
precision_recall_fscore_support(test_integer_encoded, svm_predict_usc, average='macro')

C:\Users\I515580\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


(0.5065696185917783, 0.43966280457277745, 0.4348419063969498, None)

In [121]:
simpleNN_usc = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100,50, 7), random_state=1)
simpleNN_usc.fit(sentences_embeddings_usc, integer_encoded)
simpleNN_predict_usc=simpleNN_usc.predict(test_embeddings_usc)
precision_recall_fscore_support(test_integer_encoded, simpleNN_predict_usc, average='macro')

C:\Users\I515580\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


C:\Users\I515580\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


(0.4516000640641776, 0.4480301576264168, 0.43807769865115515, None)